In [2]:
import subprocess
import json
from multiprocessing import Pool
import itertools
from datetime import datetime
import pandas as pd

In [3]:
# resources = "AWS::EC2::CustomerGateway | AWS::EC2::EIP | AWS::EC2::Host | AWS::EC2::Instance | AWS::EC2::InternetGateway | AWS::EC2::NetworkAcl | AWS::EC2::NetworkInterface | AWS::EC2::RouteTable | AWS::EC2::SecurityGroup | AWS::EC2::Subnet | AWS::CloudTrail::Trail | AWS::EC2::Volume | AWS::EC2::VPC | AWS::EC2::VPNConnection | AWS::EC2::VPNGateway | AWS::EC2::RegisteredHAInstance | AWS::EC2::NatGateway | AWS::EC2::EgressOnlyInternetGateway | AWS::EC2::VPCEndpoint | AWS::EC2::VPCEndpointService | AWS::EC2::FlowLog | AWS::EC2::VPCPeeringConnection | AWS::Elasticsearch::Domain | AWS::IAM::Group | AWS::IAM::Policy | AWS::IAM::Role | AWS::IAM::User | AWS::ElasticLoadBalancingV2::LoadBalancer | AWS::ACM::Certificate | AWS::RDS::DBInstance | AWS::RDS::DBSubnetGroup | AWS::RDS::DBSecurityGroup | AWS::RDS::DBSnapshot | AWS::RDS::DBCluster | AWS::RDS::DBClusterSnapshot | AWS::RDS::EventSubscription | AWS::S3::Bucket | AWS::S3::AccountPublicAccessBlock | AWS::Redshift::Cluster | AWS::Redshift::ClusterSnapshot | AWS::Redshift::ClusterParameterGroup | AWS::Redshift::ClusterSecurityGroup | AWS::Redshift::ClusterSubnetGroup | AWS::Redshift::EventSubscription | AWS::SSM::ManagedInstanceInventory | AWS::CloudWatch::Alarm | AWS::CloudFormation::Stack | AWS::ElasticLoadBalancing::LoadBalancer | AWS::AutoScaling::AutoScalingGroup | AWS::AutoScaling::LaunchConfiguration | AWS::AutoScaling::ScalingPolicy | AWS::AutoScaling::ScheduledAction | AWS::DynamoDB::Table | AWS::CodeBuild::Project | AWS::WAF::RateBasedRule | AWS::WAF::Rule | AWS::WAF::RuleGroup | AWS::WAF::WebACL | AWS::WAFRegional::RateBasedRule | AWS::WAFRegional::Rule | AWS::WAFRegional::RuleGroup | AWS::WAFRegional::WebACL | AWS::CloudFront::Distribution | AWS::CloudFront::StreamingDistribution | AWS::Lambda::Function | AWS::NetworkFirewall::Firewall | AWS::NetworkFirewall::FirewallPolicy | AWS::NetworkFirewall::RuleGroup | AWS::ElasticBeanstalk::Application | AWS::ElasticBeanstalk::ApplicationVersion | AWS::ElasticBeanstalk::Environment | AWS::WAFv2::WebACL | AWS::WAFv2::RuleGroup | AWS::WAFv2::IPSet | AWS::WAFv2::RegexPatternSet | AWS::WAFv2::ManagedRuleSet | AWS::XRay::EncryptionConfig | AWS::SSM::AssociationCompliance | AWS::SSM::PatchCompliance | AWS::Shield::Protection | AWS::ShieldRegional::Protection | AWS::Config::ConformancePackCompliance | AWS::Config::ResourceCompliance | AWS::ApiGateway::Stage | AWS::ApiGateway::RestApi | AWS::ApiGatewayV2::Stage | AWS::ApiGatewayV2::Api | AWS::CodePipeline::Pipeline | AWS::ServiceCatalog::CloudFormationProvisionedProduct | AWS::ServiceCatalog::CloudFormationProduct | AWS::ServiceCatalog::Portfolio | AWS::SQS::Queue | AWS::KMS::Key | AWS::QLDB::Ledger | AWS::SecretsManager::Secret | AWS::SNS::Topic | AWS::SSM::FileData | AWS::Backup::BackupPlan | AWS::Backup::BackupSelection | AWS::Backup::BackupVault | AWS::Backup::RecoveryPoint | AWS::ECR::Repository | AWS::ECS::Cluster | AWS::ECS::Service | AWS::ECS::TaskDefinition | AWS::EFS::AccessPoint | AWS::EFS::FileSystem | AWS::EKS::Cluster | AWS::OpenSearch::Domain | AWS::EC2::TransitGateway | AWS::Kinesis::Stream | AWS::Kinesis::StreamConsumer | AWS::CodeDeploy::Application | AWS::CodeDeploy::DeploymentConfig | AWS::CodeDeploy::DeploymentGroup | AWS::EC2::LaunchTemplate | AWS::ECR::PublicRepository | AWS::GuardDuty::Detector | AWS::EMR::SecurityConfiguration | AWS::SageMaker::CodeRepository | AWS::Route53Resolver::ResolverEndpoint | AWS::Route53Resolver::ResolverRule | AWS::Route53Resolver::ResolverRuleAssociation | AWS::DMS::ReplicationSubnetGroup | AWS::DMS::EventSubscription | AWS::MSK::Cluster | AWS::StepFunctions::Activity | AWS::WorkSpaces::Workspace | AWS::WorkSpaces::ConnectionAlias | AWS::SageMaker::Model | AWS::ElasticLoadBalancingV2::Listener | AWS::StepFunctions::StateMachine | AWS::Batch::JobQueue | AWS::Batch::ComputeEnvironment | AWS::AccessAnalyzer::Analyzer | AWS::Athena::WorkGroup | AWS::Athena::DataCatalog | AWS::Detective::Graph | AWS::GlobalAccelerator::Accelerator | AWS::GlobalAccelerator::EndpointGroup | AWS::GlobalAccelerator::Listener | AWS::EC2::TransitGatewayAttachment | AWS::EC2::TransitGatewayRouteTable | AWS::DMS::Certificate | AWS::AppConfig::Application | AWS::AppSync::GraphQLApi | AWS::DataSync::LocationSMB | AWS::DataSync::LocationFSxLustre | AWS::DataSync::LocationS3 | AWS::DataSync::LocationEFS | AWS::DataSync::Task | AWS::DataSync::LocationNFS | AWS::EC2::NetworkInsightsAccessScopeAnalysis | AWS::EKS::FargateProfile | AWS::Route53::HostedZone"
resources = "AWS::EC2::CustomerGateway | AWS::EC2::EIP | AWS::EC2::Host | AWS::EC2::Instance | AWS::EC2::InternetGateway | AWS::EC2::NetworkAcl | AWS::EC2::NetworkInterface | AWS::EC2::RouteTable | AWS::EC2::SecurityGroup | AWS::EC2::Subnet | AWS::CloudTrail::Trail | AWS::EC2::Volume | AWS::EC2::VPC | AWS::EC2::VPNConnection | AWS::EC2::VPNGateway | AWS::EC2::RegisteredHAInstance | AWS::EC2::NatGateway | AWS::EC2::EgressOnlyInternetGateway | AWS::EC2::VPCEndpoint | AWS::EC2::VPCEndpointService | AWS::EC2::VPCPeeringConnection | AWS::IAM::Group | AWS::IAM::Policy | AWS::IAM::Role | AWS::IAM::User |  AWS::ElasticLoadBalancingV2::LoadBalancer | AWS::ACM::Certificate | AWS::RDS::DBInstance | AWS::RDS::DBSubnetGroup | AWS::RDS::DBSecurityGroup | AWS::RDS::DBSnapshot | AWS::RDS::DBCluster | AWS::RDS::DBClusterSnapshot | AWS::RDS::EventSubscription | AWS::S3::Bucket | AWS::S3::AccountPublicAccessBlock | AWS::ElasticLoadBalancing::LoadBalancer | AWS::AutoScaling::ScalingPolicy | AWS::AutoScaling::ScheduledAction | AWS::DynamoDB::Table | AWS::Lambda::Function | AWS::NetworkFirewall::Firewall | AWS::NetworkFirewall::FirewallPolicy | AWS::NetworkFirewall::RuleGroup | AWS::ECR::Repository | AWS::EKS::Cluster | AWS::Route53::HostedZone"
resource_lists = resources.split(" | ")

### 리소스 확인

In [4]:
exist_resources = []

def find_resource(resource) -> str:    
#     for item in tqdm(resource):
    proc = subprocess.Popen(
        ['aws', 'configservice', 'list-discovered-resources', '--resource-type', f'{resource}'],
    stdout = subprocess.PIPE
    )
    out, err = proc.communicate()
    
    if 'resourceType' in str(out):
        out = out.decode('utf-8')
        out = json.loads(out)
        resourceID = [out['resourceIdentifiers'][idx]['resourceId'] for idx in range(len(out['resourceIdentifiers']))]
        return [resource, resourceID]
    else:
        return 0


In [5]:
def multiprocessing(resource_lists) -> list:
    if __name__=='__main__':  
        pool = Pool(processes=6)
        resource = pool.map(find_resource, resource_lists)
        exist_resources.append(resource)
        pool.close()
        pool.join()

In [6]:
multiprocessing(resource_lists)

In [7]:
exist_resources = list(itertools.chain(*exist_resources))
exist_resources = [item for item in exist_resources if item != 0]
exist_resources

[['AWS::EC2::Instance',
  ['i-0303414e8cd5865c0',
   'i-066189a9744d6ec3a',
   'i-08614a76a021f514f',
   'i-089772e9e1906ad02']],
 ['AWS::EC2::InternetGateway', ['igw-082d2f709fb46e0c1']],
 ['AWS::EC2::NetworkAcl', ['acl-083b0ddb288659148', 'acl-09d6463116ca6874d']],
 ['AWS::EC2::NetworkInterface',
  ['eni-011b59f865f162856',
   'eni-020dc0690f3e6a20e',
   'eni-061cabcf0e433f314',
   'eni-0d62282a423a075c3']],
 ['AWS::EC2::RouteTable', ['rtb-05c6676d8dc18909b', 'rtb-0bf37f5bb938ce786']],
 ['AWS::EC2::SecurityGroup',
  ['sg-0056aa002fd81c496',
   'sg-014d3be3fb7e16872',
   'sg-053aaf7f2b8c9b204',
   'sg-087fe725d638099cf',
   'sg-0b8f402642ccea7fb']],
 ['AWS::EC2::Subnet',
  ['subnet-00411fd2c8840890e',
   'subnet-071c230d55f28bdc4',
   'subnet-08cffe1b0ae1f9b88',
   'subnet-0f3206884d9698f40']],
 ['AWS::EC2::Volume',
  ['vol-036f5e09497c71819',
   'vol-079dc0e8cf94f5762',
   'vol-0a6914dd0377cb8de',
   'vol-0cb8e382bb271a77e']],
 ['AWS::EC2::VPC', ['vpc-03e53262faff38c42', 'vpc-0692b1a

In [8]:
exist_resources = dict(zip(range(1, len(exist_resources) + 1), exist_resources))
json.dumps(exist_resources)

'{"1": ["AWS::EC2::Instance", ["i-0303414e8cd5865c0", "i-066189a9744d6ec3a", "i-08614a76a021f514f", "i-089772e9e1906ad02"]], "2": ["AWS::EC2::InternetGateway", ["igw-082d2f709fb46e0c1"]], "3": ["AWS::EC2::NetworkAcl", ["acl-083b0ddb288659148", "acl-09d6463116ca6874d"]], "4": ["AWS::EC2::NetworkInterface", ["eni-011b59f865f162856", "eni-020dc0690f3e6a20e", "eni-061cabcf0e433f314", "eni-0d62282a423a075c3"]], "5": ["AWS::EC2::RouteTable", ["rtb-05c6676d8dc18909b", "rtb-0bf37f5bb938ce786"]], "6": ["AWS::EC2::SecurityGroup", ["sg-0056aa002fd81c496", "sg-014d3be3fb7e16872", "sg-053aaf7f2b8c9b204", "sg-087fe725d638099cf", "sg-0b8f402642ccea7fb"]], "7": ["AWS::EC2::Subnet", ["subnet-00411fd2c8840890e", "subnet-071c230d55f28bdc4", "subnet-08cffe1b0ae1f9b88", "subnet-0f3206884d9698f40"]], "8": ["AWS::EC2::Volume", ["vol-036f5e09497c71819", "vol-079dc0e8cf94f5762", "vol-0a6914dd0377cb8de", "vol-0cb8e382bb271a77e"]], "9": ["AWS::EC2::VPC", ["vpc-03e53262faff38c42", "vpc-0692b1a3b27d4ea7b"]], "10":

In [65]:
df_fine_resource = pd.DataFrame.from_dict(exist_resources, orient='index', columns=['resource_type','tesource_id'])
df_fine_resource['date']=pd.datetime.now().date()
df_fine_resource.explode('tesource_id').value_counts("resource_type").to_frame("count").reset_index()


/tmp/ipykernel_17147/3422831267.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df_fine_resource['date']=pd.datetime.now().date()


,resource_type,count
0,AWS::EC2::SecurityGroup,5
1,AWS::EC2::Instance,4
2,AWS::EC2::NetworkInterface,4
3,AWS::EC2::Subnet,4
4,AWS::EC2::Volume,4
5,AWS::EC2::NetworkAcl,2
6,AWS::EC2::RouteTable,2
7,AWS::EC2::VPC,2
8,AWS::S3::Bucket,2
9,AWS::EC2::InternetGateway,1


### google
- sdk를 통해 서비스계정을 사용하라면 환경변수를 설정해야 함

In [47]:
from google.cloud import storage
from google.oauth2 import service_account


KEY_PATH = "/home/crysis/procjets/DKMS_Fine_Resource/gcp_sa_key/pj-3tier-data-9415424f663b.json"
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
client = storage.Client(credentials = credentials, project = credentials.project_id)

In [35]:
buckets = list(client.list_buckets())
bucket_name = "test_234212"

In [42]:
bucket = client.get_bucket(bucket_name)
blob_name = "resource_list.csv"
file_path = "/home/crysis/procjets/DKMS_Fine_Resource/20230310_resource_list.csv"

In [45]:
blob = bucket.blob(blob_name)
blob.upload_from_filename(file_path)

### boto3

In [4]:
import boto3
import json

REGIONS = [
    "ap-northeast-2",
    "us-east-2"
]

def get_ec2_id(region_name):
    ec2 = boto3.client('ec2', region_name=region_name)
    ec2s = ec2.describe_instances()
    ec2s_configs = ec2s['Reservations']

    return ec2s_configs

In [26]:
test = get_ec2_id(REGIONS[1])

In [28]:
df_config = pd.DataFrame(test)

In [60]:
count = df_config['Instances'].count()
test2 = pd.DataFrame.from_dict(df_config['Instances'][0], orient='index').T
for i in range(1, count):
    test2.loc[i] = df_config['Instances'][i]
    
test2

,AmiLaunchIndex,ImageId,InstanceId,InstanceType,LaunchTime,Monitoring,Placement,PrivateDnsName,PrivateIpAddress,ProductCodes,...,CpuOptions,CapacityReservationSpecification,HibernationOptions,MetadataOptions,EnclaveOptions,PlatformDetails,UsageOperation,UsageOperationUpdateTime,PrivateDnsNameOptions,MaintenanceOptions
0,0,ami-0f3c9c466bb525749,i-089772e9e1906ad02,t2.micro,2023-03-09 08:29:33+00:00,{'State': 'disabled'},"{'AvailabilityZone': 'us-east-2a', 'GroupName'...",ip-172-31-12-122.us-east-2.compute.internal,172.31.12.122,[],...,"{'CoreCount': 1, 'ThreadsPerCore': 1}",{'CapacityReservationPreference': 'open'},{'Configured': False},"{'State': 'applied', 'HttpTokens': 'optional',...",{'Enabled': False},Linux/UNIX,RunInstances,2023-03-09 08:29:33+00:00,"{'HostnameType': 'ip-name', 'EnableResourceNam...",{'AutoRecovery': 'default'}
1,0,ami-0f3c9c466bb525749,i-0303414e8cd5865c0,t2.micro,2023-03-06 13:56:39+00:00,{'State': 'disabled'},"{'AvailabilityZone': 'us-east-2a', 'GroupName'...",ip-172-31-0-134.us-east-2.compute.internal,172.31.0.134,[],...,"{'CoreCount': 1, 'ThreadsPerCore': 1}",{'CapacityReservationPreference': 'open'},{'Configured': False},"{'State': 'applied', 'HttpTokens': 'optional',...",{'Enabled': False},Linux/UNIX,RunInstances,2023-03-06 13:56:39+00:00,"{'HostnameType': 'ip-name', 'EnableResourceNam...",{'AutoRecovery': 'default'}
2,0,ami-0f3c9c466bb525749,i-066189a9744d6ec3a,t2.micro,2023-03-09 08:17:55+00:00,{'State': 'disabled'},"{'AvailabilityZone': 'us-east-2a', 'GroupName'...",ip-172-31-9-51.us-east-2.compute.internal,172.31.9.51,[],...,"{'CoreCount': 1, 'ThreadsPerCore': 1}",{'CapacityReservationPreference': 'open'},{'Configured': False},"{'State': 'applied', 'HttpTokens': 'optional',...",{'Enabled': False},Linux/UNIX,RunInstances,2023-03-09 08:17:55+00:00,"{'HostnameType': 'ip-name', 'EnableResourceNam...",{'AutoRecovery': 'default'}
3,0,ami-02238ac43d6385ab3,i-08614a76a021f514f,t2.micro,2023-03-10 06:55:43+00:00,{'State': 'disabled'},"{'AvailabilityZone': 'us-east-2a', 'GroupName'...",ip-172-31-8-110.us-east-2.compute.internal,172.31.8.110,[],...,"{'CoreCount': 1, 'ThreadsPerCore': 1}",{'CapacityReservationPreference': 'open'},{'Configured': False},"{'State': 'applied', 'HttpTokens': 'optional',...",{'Enabled': False},Linux/UNIX,RunInstances,2023-03-10 06:55:43+00:00,"{'HostnameType': 'ip-name', 'EnableResourceNam...",{'AutoRecovery': 'default'}


In [40]:
test2.to_csv('test.csv')